# Introduction to Recurrent Neural Networks

Recurrent Neural Networks work best with any kind of sequence data. This can be for example time series prediction, sequence-to-sequence learning, sentiment analysis.

## Words to Numbers
Alle Modelle brauchen eine numerische Darstellung der Inputdaten, was sich auf viele Arten machen lässt.
Zum einen lassen sich Worte, aber auch Zeichen und n-grams in Zahlen darstellen.
n-grams sind Gruppen $n$ aufeinanderfolgender Worte. Beispielsweise wird aus `The cat sat on the mat` folgende bag-of-words genannte Gruppe: `{"The", "The cat", "cat", "cat", "cat sat", "sat", "sat on", "on", "on the", "the", "the mat", "mat"}`. Für eine bag-of-words aus 1-grams wäre dies: `{"The", "cat", "sat", "on", "the", "mat"}`.
Diese Methode ist aber eher für für flache Lernverfahren geeignet, da das Konzept der bag-of-words keine Reihenfolge kennt (`{}`).
Dieser Prozess der Aufteilung eines Texts in kleinere Bestandteile wird Tokenization genannt.

Wie lässt lassen sie sich nun in Zahlen umwandeln?
### One-Hot Encoding
Zähle, welche die $n$ meisten vorkommenden Wörter im Text sind. Baue jeden Subbestandteil einen Vektor mit $n$ Elementen. Eine $1$ an Stelle $i$ des Vektors bedeutet, dass das am $i$-ten meist vorkommende Wort in diesem Bestandteil vorhanden ist.
### Word Embeddings
Sind hochdimensionale Vektoren, die eine sematische Nähe der Worte zueinander kodieren, so sollten die Vektoren beispielsweise folgende Beziehung darstellen können: `embedding("king") + embedding("female") == embedding("queen")`. Der Vorteil zum One-Hot Encoding ist, dass die Dichte an Information viel größer ist, weil der Großteil der One-Hot-Vektoren aus $0$ besteht.

Diese Vektoren können gelernt werden, z.B. als Schicht eines NN:

In [1]:
from tensorflow.keras.layers import Embedding
embedding = Embedding(1000, 64) # (maximum number of tokens as input, dimensionality of vector)
# The input to the layer is a tensor shaped (number of samples in batch, length of these sequences)

Genauso, wie sich vortrainierte Convolution-Schichten nutzen lassen, die auf ähnlichen Problemen trainiert wurden, da diese ebenso nützliche low-level-Strukturen gefunden haben. Bekannte Beispiele sind `Word2Vec` und `GloVe`.

## Weitere spezielle Schichten von RNNs

Wenn der Mensch einen Text liest, so tut er das, indem einzelne Wortgruppen im Kontext der vorangegangenen Worte gelesen werden. Das wird in RNNs so modelliert, dass der vorangegangene Output eines Teils der gesamten Inputsequenz als Input des nächsten Teils verwendet wird. Die gesamte Sequenz wird also als Loop ihrer Bestandteile verarbeitet. Sei im folgenden Code der Input ein Tensor der Form `(Zeitschritte, Input Features)`, also die gesammte Inputsequenz mit `Input Features` Elementen wird in `Zeitschritte` zeitlich aufeinander aufbauende Schritte unterteilt. In Pseudocode die Mechanik: 

In [3]:
state_t = 0

for input_t in input_sequence: # zeitlich getrennte Sequenzen in Gesamtsquenz
    output_t = activation(dot(W, input_t) + dot(U, state_t) + b)
    state_t = output_t
# W und U sind durch Backpropagation modifizierte Gewichtsmatrizen, b der Bias    

Der Output der vorangegangenen Teilsequenz wird mit einem eigenen Gewicht in den Output der jetzigen Sequenz einbezogen.

### Eine einfacht Recurrent Layer
Auf diesem Prinzip beruht auch die `SimpleRNN`-Schicht in Keras. Allerdings nimmt diese nicht eine einzelne Gesamtsequenz entgegen, sondern ein Batch dieser. Der Inputtensor hat die Form `(batch_size, timesteps, input_features)`. Der Output wiederrum `(bactch_size, output_features)`.

Wenn es um die Frage geht, ob _alle_ bisher erstellten Output in den nächsten Input einbezogen werden sollen, wird `return_sequences = True` genommen. Ansonsten wird nur der Output der letzten Teilsequenz genutzt.

Wenn mehrere `SimpleRNN`-Schichten hintereinandergeschaltet werden, dann muss jede außer die letzte mit `return_suequences = True` versehen werden.

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN

model = Sequential()
model.add(Embedding(10000, 32)) # max num tokens, dim output vec
model.add(SimpleRNN(32)) # Gesamtsequenzen des Batchs hat 32 Elemente, durch Embedding festgelegt
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                2080      
Total params: 322,080
Trainable params: 322,080
Non-trainable params: 0
_________________________________________________________________


In [9]:
model2 = Sequential()
model2.add(Embedding(10000, 32)) # max num tokens, dim output vec
model2.add(SimpleRNN(32, return_sequences = True)) # Gesamtsequenzen des Batchs hat 32 Elemente, durch Embedding festgelegt
model2.summary()
# Hier ist nun zu sehen, dass eine zusätzliche Dimension hinzugefügt wurde, da das RNN nun mehrere Outputs für alle Teil
# -sequenzen einer Gesamtsequenz weitergibt

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, None, 32)          2080      
Total params: 322,080
Trainable params: 322,080
Non-trainable params: 0
_________________________________________________________________


## LSTM
LSTM steht für 